In [ ]:
!pip install tweepy
import tweepy, json, nltk, re, operator, string
from tweepy import OAuthHandler
from nltk import bigrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
def authorise():
    consumer_key = input('consumer key: ')
    consumer_secret = input('consumer secret: ')
    access_token = input('access token: ')
    access_secret = input('access secret: ')

def process_or_store(tweet):
    json_store.append(json.dumps(tweet, indent=4))

authorise()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

json_store = []

for status in tweepy.Cursor(api.home_timeline).items(100):
    # Process a single status
    process_or_store(status._json)

In [ ]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt','via','…']

count_stop = Counter()
count_hash = Counter()
count_only = Counter()
count_bigrams = Counter()

for document in json_store:
    tweet = json.loads(document)
    terms_stop = [term for term in preprocess(tweet['text']) if term.lower() not in stop]
    terms_hash = [term for term in terms_stop if term.startswith('#')]
    terms_only = [term for term in terms_stop if not term.startswith(('#','@'))]
    terms_bigrams = bigrams(terms_stop)
    count_stop.update(terms_stop)
    count_hash.update(terms_hash)
    count_only.update(terms_only)
    count_bigrams.update(terms_bigrams)
    terms_single = set(terms_stop)
    
print(count_stop.most_common(10))
print(count_hash.most_common(10))
print(count_only.most_common(10))
print(count_bigrams.most_common(10))